# TP 1 et 2 : Accès aux données avec index 
SAM, sujet pour étudiants

date de modification : 01/02/2023

NOM: **KABONGO**

Prénom: **Ben**

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [7]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

# le nom de la table 
TABLE = "T"
print("nom de la table :", TABLE)


# le nom du fichier qui contient les données de la table
def nom_fichier(table):
    return table + ".csv"

nom de la table : T


# I. Générer un fichier

Création du fichier contenant un exemple de données.
Ce sont des données au format csv. On suppose que chaque ligne correspond à un tuple d'une table **T** ayant *n* attributs :

$$ T (a_0, a_1, a_2, ..., a_{n-1})$$

Le premier attribut $a_0$ est unique

Les attributs suivants ne sont pas uniques (ils ont des doublons). Pour l'attribut $a_1$, il y a en moyenne 2 tuples par valeur, pour $a_2$ il y a en moyenne 4 tuples par valeurs, etc. 

Les attributs sont des nombres entiers sauf le dernier qui est une chaine de caractères.


In [8]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 * 1000
nb_attributes = 7

longueur_dernier_attribut = 100
# attribut_chaine_caracteres = "".join(choice(ascii_lowercase) for i in range(longueur_dernier_attribut))
attribut_chaine_caracteres = ''.join('-' for i in range(longueur_dernier_attribut))
# print("le dernier attribut de chaque tuple est la chaine de caracètes :", attribut_chaine_caracteres)

nb_valeurs_distinctes = nb_lines


# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

# on concatène "verticalement" les colonnes pour former les tuples.
# Le dernier attribut est une chaine de caractères
b = [ ','.join(map(lambda x: str(x), e)) + f",{attribut_chaine_caracteres}\n" for e in zip(*a)]

# on stocke les données dans un fichier
fichier = nom_fichier(TABLE)
with open(fichier, "w") as f:
    f.write(''.join(b))

print("données créées dans le fichier :", fichier)

données créées dans le fichier : T.csv


On affiche le début et la fin du fichier et son nombre de lignes ( = card(T))

In [9]:
%%bash -s "$fichier"
echo "debut de $1 :"
head -n 2 $1
echo
echo "fin de $1 : "
tail -n 2 $1
echo
echo "size (lines) :"
wc -l $1

debut de T.csv :
610466,1491367,834008,320529,159546,90830,45493,----------------------------------------------------------------------------------------------------
3385869,280258,767678,318746,120970,54031,57465,----------------------------------------------------------------------------------------------------

fin de T.csv : 
274037,1606286,1015781,555954,131214,26550,45978,----------------------------------------------------------------------------------------------------
1504608,311812,1043425,554732,294812,135593,21355,----------------------------------------------------------------------------------------------------

size (lines) :
5000000 T.csv


# II. Lecture séquentielle

On définit un *iterateur* pour lire séquentiellement chaque ligne de la table stockée entièrement dans un seul fichier. Le mot python *yield* facilite la définition d'un itérateur.

Cet itérateur est invoqué pour lire la table et appliquer un filtre.


In [10]:
def lecture_sequentielle(table):
    fichier = nom_fichier(table)
    with open(fichier, "r") as f:
        for i, line in enumerate(f):
            yield i, line

def filtrer_table(table, valeur_recherchee):
    for i, line in lecture_sequentielle(table):
        a = int(line.split(',')[0])
        if a == valeur_recherchee :
            print(f"ligne {i} :", line.strip())



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table(TABLE, s)
print("done in", round(time.time() - t1, 3), "s")

valeur recherchée : 32659
ligne 2643479 : 32659,531820,51018,535394,158963,100525,37674,----------------------------------------------------------------------------------------------------
done in 4.973 s


# III. Découper une table en pages

On organise les données en pages. 
Pour faciliter le TP, chaque page est représentée par un "petit" fichier mais en réalité une page serait un bloc d'un fichier.

Dans la suite du TP, on accédera toujours aux pages.
Le fichier créé initialement, contenant tous les tuples, ne sera plus utilisé.

In [11]:
def page_dir_name(table):
    return table + "_pages"

print("les pages sont stockées dans le dossier", page_dir_name(TABLE) )

def decoupe_table_en_pages(table, nb_tuple_par_page):
  page_dir = page_dir_name(table)

  # vider le dossier qui contiendra les pages
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  # lire le fichier contenant tous les tuples
  p=0
  lines = []
  for i, line in lecture_sequentielle(table):
    lines.append(line)
    if (i+1) % nb_tuple_par_page == 0:
      
      # créer une page
      p += 1
      with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p)


decoupe_table_en_pages(TABLE, nb_tuple_par_page=1000)

les pages sont stockées dans le dossier T_pages
nb pages créées : 5000


Afficher le nombre de tuples dans une page (pour quelques pages)

In [12]:
%%bash -s "$TABLE"
wc -l $1_pages/* | head -n 3

     1000 T_pages/page1
     1000 T_pages/page10
     1000 T_pages/page100


# IV. Lecture séquentielle d'une table découpée en pages

In [13]:
def lecture_sequentielle_par_page(fichier):
    """
    :param fichier: nom de la table
    """
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))

    # a faire : pour chaque page, lire ses lignes
    for ip, page in enumerate(os.listdir(page_dir)):
        # une ligne devient un tuple
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
            # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
                yield (ip, it, line)

In [14]:
def filtrer_table_par_pages(table, valeur_recherchee):
    # a faire pour chaque (numéro de page, position dans la page, tuple) obtnenu en invoquan la méthode ci dessus
    for (ip, it, line) in lecture_sequentielle_par_page(table):
        # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee
        values = line.split(',')
        att1 = int(values[0])
        if att1 == valeur_recherchee:
            print(att1)

In [15]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table_par_pages(TABLE, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 21513
21513
done in 5.61 s


# V. Lecture d'un tuple dans une page

Cette fonction retourne un tuple

In [16]:
def lecture_tuple(table, ip, it):
    """
    :param table : nom de la table
    :param ip: numéro de la page
    :param it: numéro de la ligne (du tuple)
    """
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir_name(table)))
    assert ip < nb_pages
    page = None
    for i in range(nb_pages):
        if int(os.listdir(page_dir_name(table))[i][4:]) == ip:
            page = os.listdir(page_dir_name(table))[i]
            print('Page', ip, end=' ')
            break
    with open(page_dir + "/" + page, 'r') as filep:
        print('Ligne', it)
        return filep.readlines()[it]

t1 = time.time()
print(lecture_tuple(TABLE, 123, 456))
print("done in", round((time.time() - t1)*1000, 1), "ms")

Page 123 Ligne 456
4055489,1954698,661690,556430,225410,9871,61075,----------------------------------------------------------------------------------------------------

done in 11541.3 ms


# VI. Créer un index

## VI. 1. Créer un index unique pour l'attribut $a_0$

On sait que $a_0$ est unique. 
Une entrée associe une *clé* à une *valeur*.
*   La *clé* est la valeur du 1er attribut. 
*   La *valeur* est un **rowid** formé des informations (page, position)



In [17]:
def creation_index_unique(table):
    """
    :param table : nom de la table
    :return dictionnaire d'index unique
    """
    index = {}
    page_dir = page_dir_name(table)
    for page in os.listdir(page_dir):
        ip = int(page[4:])
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
                index[int(line.split(',')[0])] = (ip, it)
    return index

t1 = time.time()
INDEX_UNIQUE_a0 = creation_index_unique(TABLE)
print("done in", round(time.time() - t1, 3), "s")

done in 7.79 s


In [18]:
# vérifier l'index unique
s = np.random.randint(nb_valeurs_distinctes)
print(s, INDEX_UNIQUE_a0[s])

68267 (1573, 907)


## VI. 2. Créer un index non unique pour l'attribut $a_i$

On donne un nom de table et le numéro $i$ de l'attribut $a_i$ de la table.

In [19]:
def creation_index(table, numero_attribut):
    """
    :param table : nom de la table
    :param numero_attribut
    :return dictionnaire d'index sur l'attribut
    """
    index = {}
    page_dir = page_dir_name(table)
    for page in os.listdir(page_dir):
        ip = int(page[4:])
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
                key = int(line.split(',')[numero_attribut])
                rowids = index[key] if key in index else []
                index[key] = rowids + [(ip, it)]
    return index

t1 = time.time()
INDEX_a2 = creation_index(TABLE, 2)
print("done in", round(time.time() - t1, 3), "s")

done in 49.381 s


In [20]:
# vérifier l'index
s = np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)
for ri in INDEX_a2[s]:
    if ri is None: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))

valeur recherchée : 10381
Page 1928 Ligne 733
(1928, 733) : 2563634,124692,10381,222045,41703,64856,24435,----------------------------------------------------------------------------------------------------

Page 1131 Ligne 963
(1131, 963) : 1978082,1198637,10381,623471,164655,68556,32123,----------------------------------------------------------------------------------------------------

Page 2221 Ligne 280
(2221, 280) : 3030775,1148833,10381,526013,305148,114810,71905,----------------------------------------------------------------------------------------------------

Page 1990 Ligne 450
(1990, 450) : 609636,826308,10381,372947,278580,111206,68481,----------------------------------------------------------------------------------------------------

Page 1844 Ligne 670
(1844, 670) : 1534107,1519004,10381,519081,152614,132978,64494,----------------------------------------------------------------------------------------------------



# VII. Accès par index

## VII. 1. Accès ciblé 



### VII. 1. 1. Index unique scan. 
Accès pour rechercher le tuple dont l'attribut indexé a une valeur donnée (on suppose que l'attribut est unique)

In [21]:
def acces_par_index_unique(index_unique, table, valeur_recherchee):
    """
    :param index_unique
    :param table
    :param valeur_recherche
    :return tuple (ip, it)
    """
    return index_unique.get(valeur_recherchee, None)

s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
ri = acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)
if ri is not None:
    print(ri, ':', lecture_tuple(TABLE, *ri))
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 55942
Page 1666 Ligne 487
(1666, 487) : 55942,2124923,874982,525114,50318,15825,70033,----------------------------------------------------------------------------------------------------

done in 9021.1 ms


### VII. 1. 2. Index scan
Accès pour rechercher les tuples dont l'attribut indexé a une valeur donnée. On suppose que l'attribut n'est pas unique et que plusieurs tuples sont retrouvés

In [22]:
def acces_par_index(index, table, valeur_recherchee):
    """
    :param index 
    :param table
    :param valeur_recherchee
    :return list((ip, it))
    """
    return index.get(valeur_recherchee, None)

s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
rowid = acces_par_index(INDEX_a2, TABLE, s)
for ri in rowid:
    if ri is None: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 51492
Page 4323 Ligne 371
(4323, 371) : 2708457,1659984,51492,567166,216353,117429,17659,----------------------------------------------------------------------------------------------------

done in 10045.65 ms


## VII. 2. Accès par intervalle 
Index range scan



### VII. 2. 1. Accès par intervalle sur un attribut unique
Accès pour rechercher les tuples dont l'attribut indexé est unique et a une valeur comprise dans un intervalle donné.

In [23]:
def acces_intervalle_par_index_unique(index_unique, table, borne_inf, borne_sup):
    """
    :param index_unique
    :param table
    :param borne_inf
    :param borne_sup
    :return list((ip, it))
    """
    rowid = []
    for i in range(borne_inf, borne_sup + 1):
        rowid.append(index_unique.get(i, None))
    return rowid

s = np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)

t1 = time.time()
rowid = acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, s, s+10)
for ri in rowid:
    if ri is None: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 11913
Page 731 Ligne 157
(731, 157) : 11913,2375559,527065,43972,11853,71572,67045,----------------------------------------------------------------------------------------------------

Page 1866 Ligne 632
(1866, 632) : 11914,2405299,1098437,488620,268802,96497,2250,----------------------------------------------------------------------------------------------------

Page 917 Ligne 837
(917, 837) : 11915,1472830,647388,470946,309754,6601,36256,----------------------------------------------------------------------------------------------------

Page 3311 Ligne 540
(3311, 540) : 11916,1178380,270491,602432,47622,133631,24717,----------------------------------------------------------------------------------------------------

Page 2826 Ligne 377
(2826, 377) : 11917,229143,35833,576948,108398,50959,46150,----------------------------------------------------------------------------------------------------

Page 213 Ligne 420
(213, 420) : 11918,957572,559504,171266,52791,215

### VII. 2. 2. Accès par intervalle sur un attribut NON unique
Accès pour rechercher les tuples dont l'attribut indexé n'est **pas** unique et a une valeur comprise dans un intervalle donné.

In [24]:
def acces_intervalle_par_index(index, table, borne_inf, borne_sup):    
    """
    :param index
    :param table
    :param borne_inf
    :param borne_sup
    :return list((ip, it))
    """
    rowid = []
    for i in range(borne_inf, borne_sup + 1):
        rids = index.get(i, None)
        if rids is not None:
            rowid = rowid + [*rids]
    return rowid

s = np.random.randint(nb_valeurs_distinctes / 4)
print("valeur recherchée :", s)

t1 = time.time()
rowid = acces_intervalle_par_index(INDEX_a2, TABLE, s, s + 5)
for ri in rowid:
    if ri is None: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 17891
Page 4740 Ligne 701
(4740, 701) : 2445261,662523,17891,25841,53887,40869,38766,----------------------------------------------------------------------------------------------------

Page 967 Ligne 836
(967, 836) : 3117071,1727106,17891,490847,105507,20473,72317,----------------------------------------------------------------------------------------------------

Page 1231 Ligne 93
(1231, 93) : 3643839,1117568,17891,398114,197844,104527,38294,----------------------------------------------------------------------------------------------------

Page 1015 Ligne 125
(1015, 125) : 4308450,699442,17891,496081,58363,34516,792,----------------------------------------------------------------------------------------------------

Page 2832 Ligne 159
(2832, 159) : 4854321,471545,17891,23120,306222,27982,32448,----------------------------------------------------------------------------------------------------

Page 907 Ligne 678
(907, 678) : 3281970,632473,17891,572290,139661

# VIII. Mise à jour de données

## VIII. 1. Sélectionner un tuple et modifier un de ses attributs

### VIII. 1. 1. Modification d'un seul tuple

On donne une valeur *v* de l'attribut clé $a_0$. Ajouter 1 à l'attribut $a_1$. Cela correspond à l'instruction

<center>update T
set a1 = a1+1
where a0 = *v*</center>

Après la modification, accéder aux données pour vérifier que le tuple a bien été modifié. Par exemple, invoquer la fonction
acces_par_index_unique(index, table, v)



In [25]:
def _incremente_unique(table, ip, it, numero_attribut):
    """
    modifie une ligne d'une page en incrémentant un attribut
    :param table : nom de la table
    :param ip : numéro de la page
    :param it : numéro de la ligne
    :param numero_attribut
    """
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir))
  
    page = None
    for i in range(nb_pages):
        if int(os.listdir(page_dir)[i][4:]) == ip:
            page = os.listdir(page_dir)[i]
            print(page)
            break
            
    file = open(page_dir + "/" + page, "r+")
    lines = file.readlines()
    atts = lines[it].split(',')
    atts[numero_attribut] = str( int(atts[numero_attribut]) + 1 )
    lines[it] = ','.join(atts)
    file.seek(0)
    file.writelines(lines)
    file.close()

In [26]:
def incremente_unique(index, table, v, numero_attribut=1):
    """
    incrémentation d'un attribut avec index unique
    :param index : dictionnaire d'index
    :param table : nom de la table
    :param v: valeur d'index pour laquelle modifier la ligne
    :param numero_attribut : numero de l'attribut à incrémenter
    """
    ip, it = index[v]
    _incremente_unique(table, ip, it, numero_attribut)
    
s = np.random.randint(nb_valeurs_distinctes / 4)
print('Valeur de la clé :', s)
print('\nLecture du tuple avant modification :')
print(lecture_tuple(TABLE, *INDEX_UNIQUE_a0[s]))

# modification
incremente_unique(INDEX_UNIQUE_a0, TABLE, s)

print('\nLecture du tuple après modification :')
print(lecture_tuple(TABLE, *INDEX_UNIQUE_a0[s]))

Valeur de la clé : 6140

Lecture du tuple avant modification :
Page 520 Ligne 460
6140,533403,20645,178053,66966,15456,59325,----------------------------------------------------------------------------------------------------

page520

Lecture du tuple après modification :
Page 520 Ligne 460
6140,533404,20645,178053,66966,15456,59325,----------------------------------------------------------------------------------------------------



### VIII. 1. 2. Modification de plusieurs tuples

On donne une valeur *v* de l'attribut $a_2$ qui n'est pas unique. Ajouter 1 à l'attribut $a_3$.

update T set a3 = a3+1 where a1=v



In [27]:
def _incremente(table, rowid, numero_attribut):
    """
    modifie une liste de rowid d'une page en incrémentant un attribut
    :param table : nom de la table
    :param rowid : list((ip, it))
    :param numero_attribut
    """
    # clé : pages
    # values : liste des tuples de la page
    rowid_dict = {}
    for (ip, it) in rowid:
        if not ip in rowid_dict:
            rowid_dict[ip] = []
        rowid_dict[ip] = rowid_dict[ip] + [it]
    
    for ip in rowid_dict:
        page_dir = page_dir_name(table)
        nb_pages = len(os.listdir(page_dir))
    
        page = None
        for i in range(nb_pages):
            if int(os.listdir(page_dir)[i][4:]) == ip:
                page = os.listdir(page_dir)[i]
                print(page)
                break
        
        file = open(page_dir + "/" + page, "r+")
        lines = file.readlines()
        
        # modification des tuples de la page concernés
        for it in rowid_dict[ip]:
            atts = lines[it].split(',')
            atts[numero_attribut] = str( int(atts[numero_attribut]) + 1 )
            lines[it] = ','.join(atts)
            
        file.seek(0)
        file.writelines(lines)
        file.close()

In [28]:
def incremente(index, table, v, numero_attribut=3):
    """
    incrémentation d'un attribut avec index
    :param index : dictionnaire d'index
    :param table : nom de la table
    :param v: valeur d'index pour laquelle modifier la ligne
    :param numero_attribut : numero de l'attribut à incrémenter
    """
    rowid = index[v]
    _incremente(table, rowid, numero_attribut)

    
s = np.random.randint(nb_valeurs_distinctes / 4)
print('Valeur de la clé :', s)

print('\nLecture du tuple avant modification :')
for ri in INDEX_a2[s]:
    print(lecture_tuple(TABLE, *ri))

# modification
incremente(INDEX_a2, TABLE, s)

print('\nLecture du tuple après modification :')
for ri in INDEX_a2[s]:
    print(lecture_tuple(TABLE, *ri))

Valeur de la clé : 5178

Lecture du tuple avant modification :
Page 2989 Ligne 68
4750077,1845776,5178,433875,145542,62895,50307,----------------------------------------------------------------------------------------------------

Page 3235 Ligne 315
3063009,2433071,5178,451703,211335,135387,19858,----------------------------------------------------------------------------------------------------

Page 152 Ligne 543
61561,2334364,5178,287011,65760,7003,10064,----------------------------------------------------------------------------------------------------

page2989
page3235
page152

Lecture du tuple après modification :
Page 2989 Ligne 68
4750077,1845776,5178,433876,145542,62895,50307,----------------------------------------------------------------------------------------------------

Page 3235 Ligne 315
3063009,2433071,5178,451704,211335,135387,19858,----------------------------------------------------------------------------------------------------

Page 152 Ligne 543
61561,2334364

## VIII. 2. Modifier l'index en conséquence lorsque l'attibut modifié est indexé
Par exemple, si $a_3$ est indexé, la mise à jour de la question précédente implique d'actualiser l'index sur $a_3$  pour que les rowid associés à l'ancienne valeur de $a_3$  soient associés à la nouvelle valeur de $a_3$ .

In [29]:
INDEX_a3 = creation_index(TABLE, 3)

In [30]:
def incremente_index(index, table, v, numero_attribut=2):
    """
    incrémentation d'un attribut clé d'index 
    :param index : dictionnaire d'index
    :param table : nom de la table
    :param v: valeur d'index pour laquelle modifier la ligne
    :param numero_attribut : numero de l'attribut à incrémenter
    """
    # l'attribut modifié est l'attribut indexé
    # on utilise des index non uniques
    rowid = index[v]
    index[v] = []
    if index.get(v + 1, None) is None:
        index[v + 1] = []
    index[v + 1] = index[v + 1] + rowid
    
    _incremente(table, rowid, numero_attribut)
        
    return index
    
    
s = np.random.randint(nb_valeurs_distinctes / 4)
print('Valeur de la clé :', s)

print('\nLecture du tuple avant modification :')
for ri in INDEX_a3[s]:
    print(lecture_tuple(TABLE, *ri))

# modification
INDEX_a3_2 = incremente_index(INDEX_a3, TABLE, s)

print('\nLecture du tuple après modification pour la valeur :', s)
for ri in INDEX_a3_2[s]:
    print(lecture_tuple(TABLE, *ri))

print('\nLecture du tuple après modification pour la valeur :', s + 1)
for ri in INDEX_a3_2[s + 1]:
    print(lecture_tuple(TABLE, *ri))

Valeur de la clé : 1130

Lecture du tuple avant modification :
Page 2736 Ligne 147
3224560,585275,1134239,1130,26970,51957,15540,----------------------------------------------------------------------------------------------------

Page 1413 Ligne 621
3810631,2398656,297589,1130,22809,155127,22425,----------------------------------------------------------------------------------------------------

Page 4080 Ligne 56
1409116,2256039,1204550,1130,16937,28844,77368,----------------------------------------------------------------------------------------------------

Page 4954 Ligne 268
3162151,1546521,934498,1130,140181,135906,8228,----------------------------------------------------------------------------------------------------

Page 81 Ligne 588
4085695,749013,937384,1130,28066,76344,52568,----------------------------------------------------------------------------------------------------

Page 77 Ligne 11
897445,615234,164480,1130,46408,112937,59519,------------------------------------

# IX. Persistence

Dans cette partie, on veut rendre les index persistents en les stockant dans des pages. Cela permet d'utiliser les index plus efficacement sans les recréer entièrement.

## IX. 1. Stockage d'un index unique

Proposer une solution pour stocker les entrées triées d'un index unique dans plusieurs pages avec une taille de page fixée (500 entrées par page,  soit 500 clés + 500 rowid).

In [31]:
def index_unique_dir_name(fichier):
    return fichier.split('.')[0] + "_unique_index"

def decoupe_index_unique_en_pages(fichier, nb_tuple_par_page=500):
    index_dir = index_unique_dir_name(fichier)
    index = creation_index_unique(fichier)
    print("index dans :", index_dir)
    if(os.path.exists(index_dir)):
        sh.rmtree(index_dir)
    os.makedirs(index_dir, exist_ok=True)
    
    items = list(index.items())
    items.sort()
    nb_i = len(items)
    
    nb_t = 0
    index_n = 0
    for i in range(nb_tuple_par_page, nb_i, nb_tuple_par_page):
        index_n += 1
        a = items[i - nb_tuple_par_page : i]
        lines = list(map(lambda x: ','.join([str(x[0]), str(x[1][0]), str(x[1][1])]) + '\n', a))
        # nouveau fichier
        with open(index_dir + "/index" + str(index_n), 'w') as fi:
            fi.write(''.join(lines))
        nb_t += nb_tuple_par_page
    # prise en compte des index restants -> page non pleine
    if nb_t < nb_i:
        index_n += 1
        a = items[nb_t :]
        lines = list(map(lambda x: ','.join([str(x[0]), str(x[1][0]), str(x[1][1])]) + '\n', a))
        # nouveau fichier
        with open(index_dir + "/index" + str(index_n), 'w') as fi:
            fi.write(''.join(lines))
        nb_t = nb_i
        
    print('Nombre de pages d index :', index_n)

decoupe_index_unique_en_pages(TABLE)

index dans : T_unique_index
Nombre de pages d index : 10000


## IX. 2. Stockage d'un index non unique
Proposer une solution pour stocker les entrées triées d'un index non unique dans plusieurs pages avec une taille de page fixée. Dans une page, le total du nombre  de clés + le nombre de rowid ne peut pas dépasser 1000.

In [32]:
def index_dir_name(fichier, numero_attribut):
    return fichier.split('.')[0] + "_" + str(numero_attribut) + "_index"

def rowid_to_line(key, values):
    # tranforme des rowid en une ligne à écrire
    # le nombre de valeurs est considéré inférieur à nb_rowid_par_tuple
    # renvoie la ligne
    line = [str(key)] + list(map(lambda v: str(v)[1:-1], values))
    line = ';'.join(line) + '\n'
    return line

def decoupe_index_en_pages(fichier, numero_attribut, nb_tuple_par_page=500, nb_rowid_par_tuple=1000):
    
    index = creation_index(fichier, numero_attribut)
    items = list(index.items())
    items.sort()

    lines = [] # buffer des lignes à écrire
    
    # récupération des lignes
    for key, values in items:
        
        if len(values) <= nb_rowid_par_tuple:
            # le nombre de rowids peut tenir en une ligne
            line = rowid_to_line(key, values) 
            lines.append(line)
            
        else:
            # le nombre de rowids est grand, on les écrits par paquets de 1000
            for step in range(nb_rowid_par_tuple, len(values) + 1, nb_rowid_par_tuple):
                line = rowid_to_line(key, values[step - nb_rowid_par_tuple : step])
                lines.append(line)
                
            # il reste encore des rowids à écrire
            diff = len(values) % nb_rowid_par_tuple
            if diff > 0:
                line = rowid_to_line(key, values[-diff:]) 
                lines.append(line)
        
        
    index_dir = index_dir_name(fichier, numero_attribut)
    print("index dans :", index_dir)
                
    if(os.path.exists(index_dir)):
        sh.rmtree(index_dir)
    os.makedirs(index_dir, exist_ok=True)    
    
    nb_t = 0    # numéro du tuple
    index_n = 0 # numéro de la page d'index
    
    # écriture des lignes
    while len(lines) > 0:
        # numéro de la page d'index
        index_n += 1
            
        # nouveau fichier : écrire au maximum nb_tuple_par_page tuples
        nb_tuples_ecrits = min( nb_tuple_par_page, len(lines) )
        
        with open(index_dir + "/index" + str(index_n), 'w') as fi:
            fi.write(''.join(lines[:nb_tuples_ecrits]))
        nb_t += nb_tuples_ecrits
            
        # avancement du pointeur d'écriture
        if len(lines) > nb_tuples_ecrits:
            lines = lines[nb_tuples_ecrits:]
        else: # on a fini d'écrire
            lines = []
        
    print('Nombre de pages d index :', index_n)

decoupe_index_en_pages(TABLE, 2)

index dans : T_2_index
Nombre de pages d index : 2455


## IX. 3. Adapter en conséquence les opérations de modification de l'index

### IX. 3. 1. Sélection d'une unique valeur

In [33]:
def selection_par_index_unique(fichier, valeur_recherchee, nb_tuple_par_page=500):
    index_dir = index_unique_dir_name(fichier)
    index_n = (valeur_recherchee // nb_tuple_par_page) + 1
    
    ip = -1
    it = -1
    with open(index_dir + "/index" + str(index_n), 'r') as fi:
        for line in fi.readlines():
            values = line.split(',')
            key = int(values[0])
            if key == valeur_recherchee:
                ip = int(values[1])
                it = int(values[2]) 
                break
                
    return (ip, it)

In [34]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

ip, it = selection_par_index_unique(TABLE, s)
print('Page :', ip)
print('Tuple :', it)
if ip != -1:
    print(lecture_tuple(TABLE, ip, it))

valeur recherchée : 66828
Page : 2965
Tuple : 237
Page 2965 Ligne 237
66828,924229,964792,263783,230860,54808,37525,----------------------------------------------------------------------------------------------------



In [35]:
def selection_par_index(fichier, numero_attribut, valeur_recherchee, 
                        nb_tuple_par_page=500, nb_rowid_par_tuple=1000):
    
    def flambda(v):
        ip, it = v.split(',')
        return (int(ip), int(it))
    
    index_dir = index_dir_name(fichier, numero_attribut)
    nb_pages = len(os.listdir(index_dir))
    
    # index à partir duquel on est suceptible de trouver la valeur cherchée
    index_n = 1
    
    rowid = []
    # nécessité de probablement boucler sur les pages d'index
    find = False # indique si la valeur a été trouvée
    stop_read = False # indique si on doit arrêter de lire
    
    while index_n <= nb_pages:
        
        with open(index_dir + "/index" + str(index_n), 'r') as fi:
            
            for line in fi.readlines():
                values = line.split(';')
                key = int(values[0])
                
                # test de correspondance de la clé
                if key == valeur_recherchee:
                    rowid.extend(list(map(flambda, values[1:])))
                    print(key)
                    
                    find = True
                    
                    # si le nombre d'élément de la ligne est inférieur à nb_rowid_par_tuple
                    # il faut arrêter de lire
                    # en effet, il n'y a donc plus de valeur qui nous intéresse
                    if len(values[1:]) < nb_rowid_par_tuple:
                        stop_read = True
                        break
        
        if stop_read:
            break
        
        # si on arrête pas de lire -> on passe à la page suivante
        index_n += 1
                
    return rowid

In [36]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

rowid = selection_par_index(TABLE, 2, s)
for ri in rowid:
    print(ri , ':', lecture_tuple(TABLE, *ri))

valeur recherchée : 21560
21560
Page 55 Ligne 404
(55, 404) : 553958,2083264,21560,65554,40142,148793,38533,----------------------------------------------------------------------------------------------------

Page 2014 Ligne 701
(2014, 701) : 3878037,347405,21560,389389,183012,7700,37746,----------------------------------------------------------------------------------------------------



### IX. 3. 2. Sélection d'une plage de valeurs

In [37]:
def selection_plage_par_index_unique(fichier, borne_inf, borne_sup, nb_tuple_par_page=500):
    index_dir = index_unique_dir_name(fichier)
    rowid = []
    
    index_inf = (borne_inf // nb_tuple_par_page) + 1
    index_sup = (borne_sup // nb_tuple_par_page) + 1
    
    for index_n in range(index_inf, index_sup + 1):
        ip = -1
        it = -1
        with open(index_dir + "/index" + str(index_n), 'r') as fi:
            for line in fi.readlines():
                values = line.split(',')
                key = int(values[0])
                if borne_inf <= key <= borne_sup :
                    ip = int(values[1])
                    it = int(values[2])
                    rowid.append((ip, it))
                if key > borne_sup:
                    break
        
    return rowid

In [38]:
rowid = selection_plage_par_index_unique(TABLE, 43514, 43529)
for ri in rowid:
    if ri[0] == -1: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))

Page 1737 Ligne 189
(1737, 189) : 43514,129876,1152118,586058,194900,51719,77144,----------------------------------------------------------------------------------------------------

Page 1446 Ligne 857
(1446, 857) : 43515,1710511,119603,241946,78704,143529,63801,----------------------------------------------------------------------------------------------------

Page 479 Ligne 426
(479, 426) : 43516,14431,1032797,456319,55708,65718,46416,----------------------------------------------------------------------------------------------------

Page 2439 Ligne 627
(2439, 627) : 43517,1199236,58258,108799,91540,13195,47884,----------------------------------------------------------------------------------------------------

Page 2060 Ligne 832
(2060, 832) : 43518,273290,239948,158812,56879,87999,442,----------------------------------------------------------------------------------------------------

Page 4056 Ligne 282
(4056, 282) : 43519,75810,1249139,354658,208150,38584,4262,----------------

In [39]:
def selection_plage_par_index(fichier, numero_attribut, borne_inf, borne_sup, 
                        nb_tuple_par_page=500, nb_rowid_par_tuple=1000):
    
    def flambda(v):
        ip, it = v.split(',')
        return (int(ip), int(it))
    
    index_dir = index_dir_name(fichier, numero_attribut)
    nb_pages = len(os.listdir(index_dir))
    
    # index à partir duquel on est suceptible de trouver la valeur cherchée
    index_n = 1
    
    rowid = []
    # nécessité de probablement boucler sur les pages d'index
    find = False # indique si la valeur a été trouvée
    stop_read = False # indique si on doit arrêter de lire
    
    while index_n <= nb_pages:
        
        with open(index_dir + "/index" + str(index_n), 'r') as fi:
            
            for line in fi.readlines():
                values = line.split(';')
                key = int(values[0])
                
                # test de correspondance de la clé
                if borne_inf <= key <= borne_sup:
                    rowid.extend(list(map(flambda, values[1:])))
                    print(key)
                    
                    find = True
                    
                    # on arrête de lire si on sort de l'intervalle
                    if key > borne_sup:
                        stop_read = True
                        break
        
        if stop_read:
            break
        
        # si on arrête pas de lire -> on passe à la page suivante
        index_n += 1
                
    return rowid

In [40]:
rowid = selection_plage_par_index(TABLE, 2, 4351, 4353)
for ri in rowid:
    if ri[0] == -1: continue
    print(ri, ':', lecture_tuple(TABLE, *ri))

4351
4352
4353
Page 1201 Ligne 537
(1201, 537) : 3837548,1615715,4351,235279,254192,135134,52219,----------------------------------------------------------------------------------------------------

Page 4369 Ligne 61
(4369, 61) : 4470627,57466,4351,353414,194324,125411,55454,----------------------------------------------------------------------------------------------------

Page 1649 Ligne 428
(1649, 428) : 2708611,1257521,4351,261900,120435,89122,15214,----------------------------------------------------------------------------------------------------

Page 4362 Ligne 677
(4362, 677) : 2545246,266371,4351,109096,132200,71500,40296,----------------------------------------------------------------------------------------------------

Page 824 Ligne 263
(824, 263) : 37799,2412603,4352,226581,94653,133526,68498,----------------------------------------------------------------------------------------------------

Page 1915 Ligne 524
(1915, 524) : 3244117,386974,4352,145595,238877,55607,298

### IX. 3. 3. Mise à jour d'un index

#### a. Modification d'un seul tuple

In [41]:
def incremente_persistence_unique(table, valeur, numero_attribut=1, nb_tuple_par_page=500):
    # table : nom de la table
    # valeur de l'index 
    # numéro de l'attribut à incrémenter != 0
    # att = 0 : utiliser incremente_index_persistence
    assert numero_attribut != 0, 'Pour modifier la clé unique, utilisez la fonction incremente_index_persistence'
    
    # rowid à modifier
    (ip, it) = selection_par_index_unique(table, valeur, nb_tuple_par_page)
    
    # modification du tuple si il existe
    if ip == -1: return
     
    _incremente_unique(table, ip, it, numero_attribut)
    

s = np.random.randint(nb_valeurs_distinctes / 4)
ri = selection_par_index_unique(TABLE, s)

print('Valeur de la clé :', s)
print('\nLecture du tuple avant modification :')
print(lecture_tuple(TABLE, *ri))

# modification
incremente_persistence_unique(TABLE, s)

print('\nLecture du tuple après modification :')
print(lecture_tuple(TABLE, *ri))

Valeur de la clé : 10671

Lecture du tuple avant modification :
Page 63 Ligne 201
10671,2283208,1080928,19865,14913,46732,22780,----------------------------------------------------------------------------------------------------

page63

Lecture du tuple après modification :
Page 63 Ligne 201
10671,2283209,1080928,19865,14913,46732,22780,----------------------------------------------------------------------------------------------------



#### b. Modification de plusieurs tuples

In [42]:
def incremente_persistence(table, valeur, numero_attribut_index=2, numero_attribut_a_modifier=3,
                           nb_tuple_par_page=500, nb_rowid_par_tuple=1000):
    
    assert numero_attribut_index != numero_attribut_a_modifier, 'Pour modifier la clé, utilisez la fonction incremente_index_persistence'
    
    # liste des rowid à modifier
    rowid = selection_par_index(table, numero_attribut_index, valeur,
                                nb_tuple_par_page, nb_rowid_par_tuple)
    
    _incremente(table, rowid, numero_attribut_a_modifier)
    
s = np.random.randint(nb_valeurs_distinctes / 4)
print('Valeur de la clé :', s)

rowid = selection_par_index(TABLE, 2, s)

print('\nLecture du tuple avant modification :')
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

# modification
incremente_persistence(TABLE, s)

print('\nLecture du tuple après modification :')
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

Valeur de la clé : 8107
8107

Lecture du tuple avant modification :
Page 3577 Ligne 102
2207675,1077769,8107,133255,4034,42153,36240,----------------------------------------------------------------------------------------------------

Page 3763 Ligne 478
3558376,603401,8107,374786,302100,126372,12417,----------------------------------------------------------------------------------------------------

Page 3788 Ligne 270
219282,146087,8107,293554,207272,48841,68221,----------------------------------------------------------------------------------------------------

Page 2724 Ligne 123
3640551,419499,8107,230771,236433,49325,25125,----------------------------------------------------------------------------------------------------

8107
page3577
page3763
page3788
page2724

Lecture du tuple après modification :
Page 3577 Ligne 102
2207675,1077769,8107,133256,4034,42153,36240,----------------------------------------------------------------------------------------------------

Page 3763 Lign

#### c. Modification d'un attribut indexé (non unique)

In [43]:
def incremente_index_persistence(table, valeur, numero_attribut=2, 
                                nb_tuple_par_page=500, nb_rowid_par_tuple=1000):
    
    rowid = selection_par_index(table, numero_attribut, valeur,
                                nb_tuple_par_page, nb_rowid_par_tuple)
    
    _incremente(table, rowid, numero_attribut)
    
    # modification de l'index 
    # pour des raisons de simplification : on va juste changer la clé valeur par la clé valeur + 1
    
    index_dir = index_dir_name(fichier, numero_attribut)
    nb_pages = len(os.listdir(index_dir))
    
    # index à partir duquel on est suceptible de trouver la valeur cherchée
    index_n = 1
    
    # nécessité de probablement boucler sur les pages d'index
    stop_read = False # indique si on doit arrêter de lire
    updated = False # indique si la page en cours de lecture a été modifiée
    
    while index_n <= nb_pages:
        
        updated = False
        
        file = open(index_dir + "/index" + str(index_n), 'r+')
        lines = file.readlines()
            
        for it, line in enumerate(lines):
            values = line.split(';')
            key = int(values[0])
                
            # test de correspondance de la clé
            if key == valeur:
                # if faut modifier la valeur de la clé
                values[0] = str( int(values[0]) + 1 )
                lines[it] = ';'.join(values)
                
                # cette page de l'index doit être réécrite
                updated = True
                    
            # si le nombre d'élément de la ligne est inférieur à nb_rowid_par_tuple
            # il faut arrêter de lire
            # en effet, il n'y a donc plus de valeur qui nous intéresse
            if len(values[1:]) < nb_rowid_par_tuple:
                stop_read = True
                break
        
        if updated:
            file.seek(0)
            file.writelines(lines)
            file.close()
        
        if stop_read:
            break
        
        # si on arrête pas de lire -> on passe à la page suivante
        index_n += 1

s = np.random.randint(nb_valeurs_distinctes / 4)
print('Valeur de la clé :', s)

rowid = selection_par_index(TABLE, 2, s)
print('\nLecture du tuple avant modification :')
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

# modification
incremente_index_persistence(TABLE, s)

rowid = selection_par_index(TABLE, 2, s)
print('\nLecture du tuple après modification pour la valeur :', s)
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

rowid = selection_par_index(TABLE, 2, s + 1)
print('\nLecture du tuple après modification pour la valeur :', s + 1)
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

Valeur de la clé : 5290
5290

Lecture du tuple avant modification :
Page 44 Ligne 623
4501536,1378801,5290,298882,184017,63502,3132,----------------------------------------------------------------------------------------------------

Page 4247 Ligne 42
1940704,1160647,5290,541410,99634,77496,48741,----------------------------------------------------------------------------------------------------

5290
page44
page4247
5290

Lecture du tuple après modification pour la valeur : 5290
Page 44 Ligne 623
4501536,1378801,5291,298882,184017,63502,3132,----------------------------------------------------------------------------------------------------

Page 4247 Ligne 42
1940704,1160647,5291,541410,99634,77496,48741,----------------------------------------------------------------------------------------------------

5291

Lecture du tuple après modification pour la valeur : 5291
Page 571 Ligne 859
2393429,1938087,5291,566032,299649,127985,32887,--------------------------------------------------

# Questions facultatives

## Index bitmap
*   Proposer un index ayant une structure "bitmap" pour l'attribut $a_5$. Idem pour l'attribut $a_6$. 
*   En utilisant les 2 index bitmap, rechercher les tuples de T tels que $a_5 = v_1$ et $a_6 = v_2$ pour deux valeurs $v_1, v_2$ appartenant au domaine de $a_5 \cap a_6$ .

In [44]:
def bitmap(index, valeur):
    # bitmap = liste des pages à charger
    # pour simplifier, on va rendre un dictionnaire
    # également on se limite aux requêtes d'égalité
    rowid = index.get(valeur, None)
    if rowid is None: return {}
    
    rowid_dict = {}
    for (ip, it) in rowid:
        if not ip in rowid_dict:
            rowid_dict[ip] = []
        rowid_dict[ip] = rowid_dict[ip] + [it]
        
    return rowid_dict

In [45]:
INDEX_a5 = creation_index(TABLE, 5)
INDEX_a6 = creation_index(TABLE, 6)

In [46]:
def bitmap_jointure(index1, valeur1, index2, valeur2):
    bitmap1 = bitmap(index1, valeur1)
    #print('Index 1', bitmap1)
    bitmap2 = bitmap(index2, valeur2)
    #print('Index 2', bitmap2)
    
    items1 = [(ip, it1) for ip, it in bitmap1.items() for it1 in it]
    items2 = [(ip, it2) for ip, it in bitmap2.items() for it2 in it]
    
    rowid = []
    for ip, it in items1:
        if not (ip, it) in items2: continue
        print('Page', ip)
        rowid.append((ip, it))
        
    return rowid

In [47]:
# pour que le test renvoie des résultats, 
# il est nécessaire de chercher les tuples où la jointure est vraie
valeur1 = 18073
print('Valeur de la clé pour a5:', valeur1)

valeur2 = 45862
print('Valeur de la clé pour a5:', valeur2)

rowid = bitmap_jointure(INDEX_a5, valeur1, INDEX_a6, valeur2)
print('Résultats')
for ri in rowid:
    print(lecture_tuple(TABLE, *ri))

Valeur de la clé pour a5: 18073
Valeur de la clé pour a5: 45862
Résultats


## Index couvrant une requête
* Illustrer les cas vus en TD.

In [50]:
import itertools

def creation_index_compose(table, attributs):
    """
    :param table : nom de la table
    :param attrs : liste des numéros d'attributs -> dans l'ordre
        
    >>> index = IndexCompose(TABLE, [3, 2, 6])
    """
    taille = len(attributs)
        
    values = {}
        
    page_dir = page_dir_name(table)
    for page in os.listdir(page_dir):
        ip = int(page[4:])
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
                split = line.split(',')
                values[(ip, it)] = [int(split[att]) for att in attributs]
                    
    all_rowid_array = np.array( list(values.keys()) )
    values_array = np.array( list(values.values()) )
    
        
    uniques = []      # valeurs uniques pour chaque attribut
    shape = []        # taille de l'index
    for i, att in enumerate(attributs):
        u = np.unique(values_array[:, i]) 
        print(u)
        uniques.append(u)
        shape.append(len(u))
        
    shape.append(1) # l'axe de stockage de la liste des rowid
    print(shape)
        
    index = np.zeros(shape, dtype=object)
        
    # toutes les combinaisons possibles des valeurs uniques
    combs = itertools.product(*uniques)

    # pour chaque combinaison, récupérer les rowid et les stocker dans l'index
    for comb in combs:
        rowid = []
        mask = np.all(values_array == comb, axis=1)
        idx = np.where(mask)[0].tolist()
        for i in idx: rowid.append(all_rowid_array[i])
        rowid.sort()
        index[comb] = rowid
        
    struct = {
        'attributs': attributs,
        'taille': taille,
        'index' : index,
        'uniques': uniques
    }
            
    return struct

In [ ]:
index_struct = creation_index_compose(TABLE, [2, 5])

[      0       1       2 ... 1249997 1249998 1249999]
[     0      1      2 ... 156247 156248 156249]
[1227003, 156250, 1]


In [ ]:
def select(index_struct, valeurs):
    """
    :param index_struct : index composé
    :param valeurs : liste des valeurs pour les attributs
    :return liste des rowid (ip, it) qui satisfont la requête
        
    SELECT * WHERE a3 = 1 AND a2 = 4
        
    >>> rowid = index.select([1, 4])
    """
    attributs = index_struct['attributs']
    taille = index_struct['taille']
    index = index_struct['index']
    uniques = index_struct['uniques']

    # récupération des cellules de l'index
    where = []
    for i, v in enumerate(valeurs):
        ui = np.where(uniques[i] == v)[0]
        
        # si la valeur n'existe pas, pas de résultat pour la requête
        if len(ui) == 0: return []

        where.append(ui[0])

    res = list(index[where].flatten()) # mise à plat des résultats
    # fusion des listes
    return list(itertools.chain(*res))